In [25]:
!pip install catboost osmnx geopy geopandas shapely scipy

In [30]:
from catboost import CatBoostRegressor
from geopy.distance import geodesic
import pandas as pd
import osmnx as ox
import geopandas as gpd
import numpy as np
from scipy.spatial import cKDTree
from shapely.geometry import Point, MultiPoint

model = CatBoostRegressor()

model.load_model('mediawise-model.cbm')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
def split_on_intervals(min_val, max_val, n):
    step = (max_val - min_val) / n
    intervals = [min_val + (step * x) for x in range(n + 1)]
    return intervals

def create_groups(x_intervals, y_intervals):
    groups = {}
    x_intervals = np.concatenate([[-np.inf], x_intervals, [np.inf]])
    y_intervals = np.concatenate([[-np.inf], y_intervals, [np.inf]])

    for x_i in range(len(x_intervals) - 1):
        for y_i in range(len(y_intervals) - 1):
            groups[f'x:{x_intervals[x_i]:.2f}-{x_intervals[x_i+1]:.2f}|y:{y_intervals[y_i]:.2f}-{y_intervals[y_i+1]:.2f}'] = 0

    return groups


def sort_on_groups(x_vals, y_vals, x_intervals, y_intervals, groups, only_vals=False):
    for x, y in zip(x_vals, y_vals):
        for x_i in range(len(x_intervals) - 1):
            for y_i in range(len(y_intervals) - 1):
                if (x_intervals[x_i] <= x < x_intervals[x_i + 1]) and (y_intervals[y_i] <= y < y_intervals[y_i + 1]):
                    groups[f'x:{x_intervals[x_i]:.2f}-{x_intervals[x_i+1]:.2f}|y:{y_intervals[y_i]:.2f}-{y_intervals[y_i+1]:.2f}'] += 1

    return list(groups.values()) if only_vals else groups

def create_dataset(config, gdf):
    x_intervals = split_on_intervals(config['min_xval'], config['max_xval'], config['x_ngroups'])
    y_intervals = split_on_intervals(config['min_yval'], config['max_yval'], config['y_ngroups'])

    groups = create_groups(x_intervals, y_intervals)

    groups_values = []
    for _, row in gdf.iterrows():
        points = np.array([[float(x['lat']), float(x['lon'])] for x in row['points']])
        group_values = sort_on_groups(points[:, 0], points[:, 1], x_intervals, y_intervals, groups.copy(), only_vals=True)
        groups_values.append(group_values)

    groups_values = np.array(groups_values)

    result = pd.DataFrame(groups_values, columns=list(groups.keys()))

    # Add additional features
    result['num_points'] = gdf['points'].apply(len)
    result['avg_lat'] = gdf['points'].apply(lambda x: np.mean([float(p['lat']) for p in x]))
    result['avg_lon'] = gdf['points'].apply(lambda x: np.mean([float(p['lon']) for p in x]))
    result['avg_azimuth'] = gdf['points'].apply(lambda x: np.mean([p['azimuth'] for p in x]))

    # # Add target audience features
    result['gender'] = gdf['gender'].map({'all': 0, 'male': 1, 'female': 2})
    result['ageFrom'] = gdf['ageFrom']
    result['ageTo'] = gdf['ageTo']
    result['age_range'] = gdf['ageTo'] - gdf['ageFrom']
    result['income'] = gdf['income'].map({'a': 1, 'b': 2, 'c': 3, 'ab': 4, 'bc': 5, 'ac': 6, 'abc': 7})

    #Add new features
    result['distance_to_center'] = gdf['distance_to_center']
    result['distance_to_metro'] = gdf['distance_to_metro']
    result['population_density'] = gdf['population_density']
    result['distance_to_shopping_center'] = gdf['distance_to_shopping_center']

    return result

In [32]:
# Load and prepare data
metro_stations = ox.features_from_place('Moscow, Russia', tags={'railway': 'station', 'station': 'subway'})
shopping_centers = ox.features_from_place('Moscow, Russia', tags={'shop': 'mall'})
moscow = ox.geocode_to_gdf('Moscow, Russia')

population_density = {
    'Центральный': 11702.67,
    'Северный': 10709.15,
    'Северо-Восточный': 14289.05	,
    'Восточный': 9743.75,
    'Юго-Восточный': 12893.76,
    'Южный': 13422.73,
    'Юго-Западный': 12890.82,
    'Западный': 9312.38,
    'Северо-Западный': 11144.78,
    'Зеленоградский': 7272.25,
    'Новомосковский': 1497.79,
    'Троицкий': 181.13
}

# Function to get centroid coordinates
def get_centroid_coords(geom):
    if geom.geom_type == 'Point':
        return (geom.y, geom.x)
    else:
        centroid = geom.centroid
        return (centroid.y, centroid.x)

# Prepare KD-trees
metro_points = np.array([get_centroid_coords(geom) for geom in metro_stations.geometry])
shopping_points = np.array([get_centroid_coords(geom) for geom in shopping_centers.geometry])
metro_tree = cKDTree(metro_points)
shopping_tree = cKDTree(shopping_points)

# Prepare spatial index
moscow = ox.geocode_to_gdf('Moscow, Russia')
moscow_sindex = moscow.sindex


prediction_cache = {}

def predict_reach(points, target_audience):
    # Convert points to a hashable type for caching
    points_key = tuple(sorted((p['lon'], p['lat']) for p in points))
    # Check if we have a cached result
    if points_key in prediction_cache:
        return prediction_cache[points_key]

    # If not in cache, compute the prediction
    df = pd.DataFrame({'points': [points], 'income': [target_audience['income']], 'gender': [target_audience['gender']], 'ageTo': [target_audience['ageTo']], 'ageFrom': [target_audience['ageFrom']] })

    df['geometry'] = df['points'].apply(lambda x: Point(float(x[0]['lon']), float(x[0]['lat'])))
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    gdf.crs = 'EPSG:4326'

    # Compute features
    red_square = (55.753930, 37.620393)
    gdf['distance_to_center'] = gdf.apply(lambda row: geodesic((row.geometry.y, row.geometry.x), red_square).kilometers, axis=1)

    coords = np.array([(point.y, point.x) for point in gdf.geometry])
    gdf['distance_to_metro'] = metro_tree.query(coords)[0]
    gdf['distance_to_shopping_center'] = shopping_tree.query(coords)[0]

    possible_matches_index = list(moscow_sindex.intersection(gdf.total_bounds))
    possible_matches = moscow.iloc[possible_matches_index]
    points_in_districts = gpd.sjoin(gdf, possible_matches, how='left', predicate='within')
    gdf['district'] = points_in_districts['name']
    gdf['population_density'] = gdf['district'].map(population_density)

    # Add target audience data
    for key, value in target_audience.items():
        gdf[key] = value

    # Create dataset for prediction
    X = create_dataset(config, gdf)

    # Make prediction
    prediction = model.predict(X)[0]

    # Cache the result
    prediction_cache[points_key] = prediction

    return prediction

config = {'min_xval':55.55, 'max_xval':55.95, 'min_yval':37.3, 'max_yval':37.9, 'x_ngroups': 33, 'y_ngroups': 33}

In [35]:
records = pd.read_json('test_data.json')

records['value'] = records.apply(lambda row: max(predict_reach(row['points'], row['targetAudience']), 0), axis=1)
records.drop(['targetAudience', 'points'], axis=1, inplace=True)

In [36]:
records.to_csv('submission.csv', sep =',', lineterminator='\n', index=False)

<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
